### Tratando os dados para regressão, clusterização e demais modelagens

In [2]:
import pandas as pd
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OrdinalEncoder, MinMaxScaler
import warnings
warnings.filterwarnings("ignore")

In [4]:
diamonds = pd.read_csv('./data/diamonds.csv')
diamonds

,Unnamed: 0,carat,cut,color,clarity,depth,table,price,x,y,z
0,1,0.23,Ideal,E,SI2,61.5,55.0,326,3.95,3.98,2.43
1,2,0.21,Premium,E,SI1,59.8,61.0,326,3.89,3.84,2.31
2,3,0.23,Good,E,VS1,56.9,65.0,327,4.05,4.07,2.31
3,4,0.29,Premium,I,VS2,62.4,58.0,334,4.20,4.23,2.63
4,5,0.31,Good,J,SI2,63.3,58.0,335,4.34,4.35,2.75
...,...,...,...,...,...,...,...,...,...,...,...
53935,53936,0.72,Ideal,D,SI1,60.8,57.0,2757,5.75,5.76,3.50
53936,53937,0.72,Good,D,SI1,63.1,55.0,2757,5.69,5.75,3.61
53937,53938,0.70,Very Good,D,SI1,62.8,60.0,2757,5.66,5.68,3.56
53938,53939,0.86,Premium,H,SI2,61.0,58.0,2757,6.15,6.12,3.74


In [9]:
# Definir os bins e labels e acrescenta a coluna 'carat range'
bins = [0.0, 0.5, 1.0, 1.5, 2.0, 2.5, 3.0, 3.5, 4.0, 4.5, 6.0]  # Adjust these ranges as needed
labels = ['0.0-0.5', '0.6-1.0', '1.1-1.5', '1.6-2.0', '2.1-2.5', '2.6-3.0', '3.1-3.5', '3.6-4.0', '4.1-4.5', 'Acima de 4.6']

# Categoriza os quilates nos bis definidos
diamonds['carat range'] = pd.cut(diamonds['carat'], bins=bins, labels=labels)

diamonds

,Unnamed: 0,carat,cut,color,clarity,depth,table,price,x,y,z,carat range
0,1,0.23,Ideal,E,SI2,61.5,55.0,326,3.95,3.98,2.43,0.0-0.5
1,2,0.21,Premium,E,SI1,59.8,61.0,326,3.89,3.84,2.31,0.0-0.5
2,3,0.23,Good,E,VS1,56.9,65.0,327,4.05,4.07,2.31,0.0-0.5
3,4,0.29,Premium,I,VS2,62.4,58.0,334,4.20,4.23,2.63,0.0-0.5
4,5,0.31,Good,J,SI2,63.3,58.0,335,4.34,4.35,2.75,0.0-0.5
...,...,...,...,...,...,...,...,...,...,...,...,...
53935,53936,0.72,Ideal,D,SI1,60.8,57.0,2757,5.75,5.76,3.50,0.6-1.0
53936,53937,0.72,Good,D,SI1,63.1,55.0,2757,5.69,5.75,3.61,0.6-1.0
53937,53938,0.70,Very Good,D,SI1,62.8,60.0,2757,5.66,5.68,3.56,0.6-1.0
53938,53939,0.86,Premium,H,SI2,61.0,58.0,2757,6.15,6.12,3.74,0.6-1.0


In [10]:
# Definição das hierarquias das categorias
cut_order = ['Fair', 'Good', 'Very Good', 'Premium', 'Ideal']
color_order = ['J', 'I', 'H', 'G', 'F', 'E', 'D']
clarity_order = ['I1', 'SI2', 'SI1', 'VS2', 'VS1', 'VVS2', 'VVS1', 'IF']
carat_order = ['0.0-0.5', '0.6-1.0', '1.1-1.5', '1.6-2.0', '2.1-2.5', '2.6-3.0', '3.1-3.5', '3.6-4.0', '4.1-4.5', 'Acima de 4.6']

# ColumnTransformer com OrdinalEncoder e MinMaxScaler
preprocessor = ColumnTransformer(
    transformers=[
        # OrdinalEncoder nas colunas categóricas com hierarquia
        ('cut', OrdinalEncoder(categories=[cut_order]), ['cut']),
        ('color', OrdinalEncoder(categories=[color_order]), ['color']),
        ('clarity', OrdinalEncoder(categories=[clarity_order]), ['clarity']),
        ('carat range', OrdinalEncoder(categories=[carat_order]), ['carat range']),
        
        # MinMaxScaler nas colunas numéricas
        ('scaler', MinMaxScaler(), ['depth', 'table', 'price', 'x', 'y', 'z'])
    ]
)

# Pipeline
pipeline = Pipeline(steps=[
    ('preprocessor', preprocessor)
])

# Transformar o dataset
diamonds_processed = pipeline.fit_transform(diamonds)

# Resultado volta como um DataFrame para facilitar a visualização
diamonds_processed_df = pd.DataFrame(diamonds_processed, columns=[
    'cut', 'color', 'clarity', 'carat', 'depth', 'table', 'price', 'x', 'y', 'z'
])

# Resultado do processamento
print(diamonds_processed_df.head())

   cut  color  clarity  carat     depth     table     price         x  \
0  4.0    5.0      1.0    0.0  0.513889  0.230769  0.000000  0.367784   
1  3.0    5.0      2.0    0.0  0.466667  0.346154  0.000000  0.362197   
2  1.0    5.0      4.0    0.0  0.386111  0.423077  0.000054  0.377095   
3  3.0    1.0      3.0    0.0  0.538889  0.288462  0.000433  0.391061   
4  1.0    0.0      1.0    0.0  0.563889  0.288462  0.000487  0.404097   

          y         z  
0  0.067572  0.076415  
1  0.065195  0.072642  
2  0.069100  0.072642  
3  0.071817  0.082704  
4  0.073854  0.086478  


In [12]:
# Salvar o DataFrame em um arquivo CSV
caminho_arquivo = "./data/cleaned.csv" 
diamonds_processed_df.to_csv(caminho_arquivo, index=False)
print(f"Arquivo salvo em: {caminho_arquivo}")

Arquivo salvo em: ./data/cleaned.csv
